In [ ]:
import numpy as np
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random

# Stage 1: Baseline Setup - Varying Pump Rate

In [ ]:
def generate_operations_file(
    input_file: str,
    output_file: str,
    num_rows: int = 590000,
    pump_rate_min: float = 5.0,
    pump_rate_max: float = 2000.0,
    mean_off_duration: int = 40,
    mean_on_duration: int = 3000,
    pump_rate_change_std: float = 200.0,
    min_off_steps: int = 20,
    min_on_steps: int = 1000,
    mean_stability_duration: int = 100
):

    with open(input_file, 'r') as f:
        lines = f.readlines()
    
    data_lines = [l.strip() for l in lines if l.strip() and not l.strip().startswith("#")]

    densities = []
    parsed_lines = []
    for l in data_lines:
        cols = l.split()
        time = float(cols[0])
        volume = float(cols[1])
        pump_rate = float(cols[2])
        fluid_name = cols[3]
        density = float(cols[4])
        ROP = float(cols[5])
        BOP_open = float(cols[6])
        bound_val_1 = float(cols[7])
        bound_val_2 = float(cols[8])
        bound_type_1 = float(cols[9])
        bound_type_2 = float(cols[10])
        DS_rot = float(cols[11])
        TemperatIn = float(cols[12])
        
        densities.append(density)
        parsed_lines.append((time, volume, pump_rate, fluid_name, density, ROP,
                             BOP_open, bound_val_1, bound_val_2,
                             bound_type_1, bound_type_2, DS_rot, TemperatIn))
    
    mean_density = np.mean(densities)
    std_density = np.std(densities)

    baseline = parsed_lines[0]
    baseline_time, baseline_volume, baseline_pump_rate, baseline_fluid_name, baseline_density, baseline_ROP, baseline_BOP_open, \
    baseline_bound_val_1, baseline_bound_val_2, baseline_bound_type_1, baseline_bound_type_2, baseline_DS_rot, baseline_TemperatIn = baseline
    
    current_pump_rate = 0.0
    state = "off"
    steps_in_current_state = 0

    off_duration = min_off_steps + int(np.random.exponential(mean_off_duration))
    on_duration = min_on_steps + int(np.random.exponential(mean_on_duration))

    steps_in_current_pump_rate = 0
    pump_rate_stability_duration = max(1, int(np.random.exponential(mean_stability_duration)))

    output_lines = []
    for i in range(num_rows):
        current_time = baseline_time 
        steps_in_current_state += 1

        if state == "off":
            current_pump_rate = 0.0
            if steps_in_current_state >= off_duration:
                state = "on"
                steps_in_current_state = 0
                on_duration = min_on_steps + int(np.random.exponential(mean_on_duration))
                current_pump_rate = np.random.uniform(50, 1100)
                steps_in_current_pump_rate = 0
                pump_rate_stability_duration = max(1, int(np.random.exponential(mean_stability_duration)))
                
        elif state == "on":
            steps_in_current_pump_rate += 1
            
            if steps_in_current_pump_rate >= pump_rate_stability_duration:
                delta = np.random.randn() * pump_rate_change_std
                new_pump_rate = current_pump_rate + delta
                new_pump_rate = np.clip(new_pump_rate, pump_rate_min, pump_rate_max)
                
                if new_pump_rate < pump_rate_min:
                    new_pump_rate = np.random.uniform(5.0, 10)
                
                current_pump_rate = new_pump_rate
                steps_in_current_pump_rate = 0
                pump_rate_stability_duration = max(1, int(np.random.exponential(mean_stability_duration)))

            if steps_in_current_state >= on_duration:
                state = "off"
                steps_in_current_state = 0
                off_duration = min_off_steps + int(np.random.exponential(mean_off_duration))
                current_pump_rate = 0.0

        current_ROP = 0.0
        current_density = np.random.normal(mean_density, std_density)
        
        line = f"{current_time}\t{baseline_volume}\t{current_pump_rate}\t{baseline_fluid_name}\t{current_density}\t{current_ROP}\t{baseline_BOP_open}\t{baseline_bound_val_1}\t{baseline_bound_val_2}\t{baseline_bound_type_1}\t{baseline_bound_type_2}\t{baseline_DS_rot}\t{baseline_TemperatIn}\n"
        output_lines.append(line)

    with open(output_file, 'w') as f:
        f.write("# Time\tVolume\tPump rate\tFluid name\tDensity\tROP\tBOP open\tBound val 1\tBound val 2\tBound type 1\tBound type 2\tDS rot\tTemperatIn\n")
        f.writelines(output_lines)

# Stage 2: Adding Drillstring Rotation

### Normal Drill String Rotation Generation

In [ ]:
def load_pump_rot_data(filename: str):
    rows = []
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            
            cols = line.split()
            if len(cols) < 13:
                continue
            # 0: Time, 1: Volume, 2: Pump rate, 11: DS_rot
            pump_rate = float(cols[2])
            ds_rot    = float(cols[11])
            rows.append((pump_rate, ds_rot))
    
    df = pd.DataFrame(rows, columns=["pump_rate", "ds_rot"])
    return df

In [ ]:
def train_ds_rot_model(
    train_file: str, 
    classifier_out="clf_rf.pkl", 
    regressor_out="reg_rf.pkl"
):

    df = load_pump_rot_data(train_file)
    
    df["rot_class"] = (df["ds_rot"] > 0).astype(int)
    X = df[["pump_rate"]].values
    y_class = df["rot_class"].values
    
    X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
        X, y_class, test_size=0.2, random_state=42
    )
    
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train_c, y_train_c)
    acc = clf.score(X_test_c, y_test_c)
    print("Classification accuracy (RF):", acc)
    
    df_nonzero = df[df["rot_class"] == 1]
    X_reg = df_nonzero[["pump_rate"]].values
    y_reg = df_nonzero["ds_rot"].values
    
    X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
        X_reg, y_reg, test_size=0.2, random_state=42
    )
    
    reg = RandomForestRegressor(n_estimators=100, random_state=42)
    reg.fit(X_train_r, y_train_r)
    r2 = reg.score(X_test_r, y_test_r)
    print("Regression R^2 score (RF):", r2)
    
    with open(classifier_out, "wb") as f:
        pickle.dump(clf, f)
    with open(regressor_out, "wb") as f:
        pickle.dump(reg, f)

    from sklearn.metrics import precision_recall_fscore_support

    probs_c = clf.predict_proba(X_test_c)[:, 1]       
    print("\nprecision / recall / F1 on validation split")
    print("thresh |  prec   rec    F1")
    for t in [0.30, 0.40, 0.60]:
        y_pred = (probs_c >= t).astype(int)
        p, r, f, _ = precision_recall_fscore_support(
            y_test_c, y_pred, average="binary", zero_division=0
        )
        print(f"{t:6.2f} | {p:5.3f}  {r:5.3f}  {f:5.3f}")

    
    print(f"Models saved to {classifier_out}, {regressor_out}")

In [ ]:
def apply_ds_rot_model(
    input_file: str, 
    output_file: str,
    classifier_file="clf_rf.pkl",
    regressor_file="reg_rf.pkl",
    max_rows: int = 590000,
    steps_between_random: int = 200, 
    random_period_length: int = 5  
):

    with open(classifier_file, "rb") as f:
        clf = pickle.load(f)
    with open(regressor_file, "rb") as f:
        reg = pickle.load(f)
    
    output_lines = []
    row_count = 0
    
    with open(input_file, 'r') as f:
        total_lines = sum(1 for _ in f)
    
    prev_pump_rate = None
    time_since_last_random = 0
    random_period_in_progress = False
    random_period_steps_left = 0
    
    ds_rot_buffer = 0.0
    
    with open(input_file, 'r') as f, tqdm(total=min(total_lines, max_rows), desc="Processing rows") as pbar:
        for line in f:
            if row_count >= max_rows:
                break
            
            line_stripped = line.strip()
            row_count += 1
            pbar.update(1)
            
            if not line_stripped or line_stripped.startswith("#"):
                output_lines.append(line)
                continue
            
            cols = line_stripped.split()
            if len(cols) < 13:
                output_lines.append(line)
                continue
            
            pump_rate = float(cols[2])
            
            if pump_rate != prev_pump_rate:
                time_since_last_random = 0
                random_period_in_progress = False
                random_period_steps_left = 0

                p_nonzero = clf.predict_proba([[pump_rate]])[0, 1]
                threshold = 0.4
                
                if pump_rate > 700:
                    ds_rot_pred = reg.predict([[pump_rate]])[0]
                else:
                    if p_nonzero > threshold:
                        ds_rot_pred = reg.predict([[pump_rate]])[0]
                    else:
                        ds_rot_pred = 0.0
                
                ds_rot_pred = max(ds_rot_pred, 0)
                ds_rot_buffer = ds_rot_pred
                prev_pump_rate = pump_rate

            else:
                
                if random_period_in_progress:
                    ds_rot_pred = ds_rot_buffer + np.random.normal(0, 10)
                    ds_rot_pred = max(ds_rot_pred, 0.0)
                    
                    random_period_steps_left -= 1
                    
                    if random_period_steps_left <= 0:
                        random_period_in_progress = False
                        ds_rot_buffer = ds_rot_pred
                        time_since_last_random = 0
                else:
                    ds_rot_pred = ds_rot_buffer
                    time_since_last_random += 1

                    if time_since_last_random >= steps_between_random:
                        random_period_in_progress = True
                        random_period_steps_left = random_period_length
                        time_since_last_random = 0

            cols[11] = f"{ds_rot_pred:.4f}"

            new_line = "\t".join(cols) + "\n"
            output_lines.append(new_line)

    with open(output_file, 'w') as f:
        f.writelines(output_lines)
    
    print(f"DS_rot updated file written to {output_file}")


### Extreme Drill String Rotation

In [ ]:
def apply_ds_rot_extreme_model(
    input_file: str, 
    output_file: str,
    classifier_file="clf_rf.pkl",
    regressor_file="reg_rf.pkl",
    max_rows: int = 590000,
    steps_between_random: int = 200,  
    random_period_length: int = 5,   
    rotation_low = 150,
    rotation_high = 400
):
    with open(classifier_file, "rb") as f:
        clf = pickle.load(f)
    with open(regressor_file, "rb") as f:
        reg = pickle.load(f)
    
    output_lines = []
    row_count = 0
    
    with open(input_file, 'r') as f:
        total_lines = sum(1 for _ in f)
    
    prev_pump_rate = None
    time_since_last_random = 0
    random_period_in_progress = False
    random_period_steps_left = 0
    
    ds_rot_buffer = 0.0
    
    with open(input_file, 'r') as f, tqdm(total=min(total_lines, max_rows), desc="Processing rows") as pbar:
        for line in f:
            if row_count >= max_rows:
                break
            
            line_stripped = line.strip()
            row_count += 1
            pbar.update(1)
            
            if not line_stripped or line_stripped.startswith("#"):
                output_lines.append(line)
                continue
            
            cols = line_stripped.split()
            if len(cols) < 13:
                output_lines.append(line)
                continue
            
            pump_rate = float(cols[2])
            
            if pump_rate != prev_pump_rate:
                time_since_last_random = 0
                random_period_in_progress = False
                random_period_steps_left = 0
                p_nonzero = clf.predict_proba([[pump_rate]])[0, 1]
                threshold = 0.4
                
                if pump_rate > 700:
                    ds_rot_pred = reg.predict([[pump_rate]])[0] + np.random.uniform(rotation_low, rotation_high)
                else:
                    if p_nonzero > threshold:
                        ds_rot_pred = reg.predict([[pump_rate]])[0] + np.random.uniform(rotation_low, rotation_high)
                    else:
                        ds_rot_pred = 0.0
                
                ds_rot_pred = max(ds_rot_pred, 0)
                ds_rot_buffer = ds_rot_pred
                prev_pump_rate = pump_rate

            else:
                if random_period_in_progress:
                    ds_rot_pred = ds_rot_buffer + np.random.normal(0, 10)
                    ds_rot_pred = max(ds_rot_pred, 0.0)
                    
                    random_period_steps_left -= 1
                    
                    if random_period_steps_left <= 0:
                        random_period_in_progress = False
                        ds_rot_buffer = ds_rot_pred
                        time_since_last_random = 0
                else:
                    ds_rot_pred = ds_rot_buffer
                    time_since_last_random += 1
                    
                    if time_since_last_random >= steps_between_random:
                        random_period_in_progress = True
                        random_period_steps_left = random_period_length
                        time_since_last_random = 0

            cols[11] = f"{ds_rot_pred:.4f}"

            new_line = "\t".join(cols) + "\n"
            output_lines.append(new_line)
    
    with open(output_file, 'w') as f:
        f.writelines(output_lines)
    
    print(f"DS_rot updated file written to {output_file}")


### Random Drill String Rotation

In [ ]:
def apply_rotation_rnd(
    input_file: str, 
    output_file: str,
    max_rows: int = 590_000,
    rotation_low: float = 100.0,
    rotation_high: float = 300.0,
    rotation_steps: int = 100
):
    output_lines = []
    row_count = 0
    with open(input_file, 'r') as f:
        total_lines = sum(1 for _ in f)
    
    current_rotation = 0.0
    steps_remaining_in_block = 0

    with open(input_file, 'r') as f, tqdm(total=min(total_lines, max_rows), desc="Applying extreme rotation") as pbar:
        for line in f:
            if row_count >= max_rows:
                break
            line_stripped = line.strip()
            row_count += 1
            pbar.update(1)
            if not line_stripped or line_stripped.startswith("#"):
                output_lines.append(line)
                continue
            
            cols = line_stripped.split()
            if len(cols) < 13:
                output_lines.append(line)
                continue

            if steps_remaining_in_block <= 0:
                current_rotation = np.random.uniform(rotation_low, rotation_high)
                steps_remaining_in_block = rotation_steps
            
            cols[11] = f"{current_rotation:.4f}"
            steps_remaining_in_block -= 1
            new_line = "\t".join(cols) + "\n"
            output_lines.append(new_line)
  
    with open(output_file, 'w') as f:
        f.writelines(output_lines)
    
    print(f"File with extreme rotation saved to {output_file}. Processed {row_count} rows.")


# Stage 3: Adding a Second Drilling Fluid and Random Switches

NOTE: In this stage it is important that when you run the physics-based model (Flowmodel.dll), a second fluid is defined in the .wel file. This is not important for the generation of this .in file, but it is required once you run the Flowmodel.dll.

In [ ]:
def get_density_stats(
    reference_file: str,
    fluid_name: str,
    fluid_col_index: int = 3,
    density_col_index: int = 4
) -> tuple[float, float]:

    densities = []
    with open(reference_file, 'r') as f:
        for line in f:
            line_stripped = line.strip()
            if not line_stripped or line_stripped.startswith("#"):
                continue
            
            cols = line_stripped.split()
            if len(cols) <= max(fluid_col_index, density_col_index):
                continue

            if cols[fluid_col_index] == fluid_name:
                try:
                    density_val = float(cols[density_col_index])
                    densities.append(density_val)
                except ValueError:
                    pass  
    return (np.mean(densities), np.std(densities))

In [ ]:
def update_fluid_and_density(
    input_operations: str,
    output_operations: str,
    fluid_a: str = "obm1",
    fluid_b: str = "Case_2a1_Fluid",
    switch_interval_min: int = 362, 
    switch_interval_max: int = 8676,
    mean_density_a: float = 1.30,
    std_density_a: float = 0.05,
    mean_density_b: float = 1.40,
    std_density_b: float = 0.05,
    density_min: float = 1.0,
    density_max: float = 2.0,
    fluid_col_index: int = 3,
    density_col_index: int = 4
):
    lines_out = []
    
    current_fluid = fluid_a 
    row_count = 0
    switch_interval = random.randint(switch_interval_min, switch_interval_max)

    with open(input_operations, 'r') as f:
        for line in f:
            line_stripped = line.strip()
            if not line_stripped or line_stripped.startswith("#"):
                lines_out.append(line)
                continue

            cols = line_stripped.split()
            if len(cols) < max(fluid_col_index, density_col_index) + 1:
                lines_out.append(line)
                continue
            
            if row_count != 0 and (row_count % switch_interval == 0):
                current_fluid = fluid_b if (current_fluid == fluid_a) else fluid_a
                switch_interval = random.randint(switch_interval_min, switch_interval_max)  

            cols[fluid_col_index] = current_fluid

            if current_fluid == fluid_a:
                new_density = np.random.normal(mean_density_a, std_density_a)
            else:
                new_density = np.random.normal(mean_density_b, std_density_b)

            new_density = max(density_min, min(new_density, density_max))

            cols[density_col_index] = f"{new_density:.5f}"

            new_line = "\t".join(cols) + "\n"
            lines_out.append(new_line)

            row_count += 1

    with open(output_operations, 'w') as f:
        f.writelines(lines_out)

The other fluid I use is a fluid found in Case_2a1. So, fluid Case_2a_Fluid.

In [ ]:
mean_dens_a, std_dens_a = get_density_stats(
    "../input_files/_test_rotation_ext/Case_5m_rot_operation.in",
    fluid_name="obm1",
    fluid_col_index=3,
    density_col_index=4
)
print(f"obm1 stats => mean: {mean_dens_a}, std: {std_dens_a}")

mean_dens_b, std_dens_b = get_density_stats(
    "../input_files/case_2a1/Case_2a1_Drill_only_operations.in",
    fluid_name="Case_2a1_Fluid",
    fluid_col_index=3,
    density_col_index=4
)
print(f"Case_2a1_Fluid stats => mean: {mean_dens_b}, std: {std_dens_b}")

update_fluid_and_density(
    input_operations="../input_files/_test_rotation_ext/Case_5m_rot_operation.in",
    output_operations="../input_files/_test_rot_drill_fluid/Case_5m_rot_operation.in",
    fluid_a="obm1",
    fluid_b="Case_2a1_Fluid",
    switch_interval_min = 362, 
    switch_interval_max = 8676, 
    mean_density_a=mean_dens_a, 
    std_density_a=std_dens_a,
    mean_density_b=mean_dens_b,  
    std_density_b=std_dens_a,
    density_min=1.0,
    density_max=2.0,
    fluid_col_index=3, 
    density_col_index=4
)